In [ ]:
## Facebook data

import pandas as pd
from google.cloud import storage
from datetime import datetime, timedelta
import os

def prep_excel_to_parquet(event, context):
    bucket_name = 'bucket_name'
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    for blob in blobs:
        if blob.name.endswith('.xlsx'):
            file_name = os.path.basename(blob.name)
            modified_time = blob.updated.replace(tzinfo=None)

            ## Get the latest file only
            if modified_time >= datetime.now() - timedelta(hours=24):
                tmp_path = f"/tmp/{file_name}"
                download_blob(bucket_name, file_name, tmp_path)

def download_blob(bucket_name, blob_name, tmp_path,request=None):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(tmp_path)
    clean_upload_data(tmp_path)

def clean_upload_data(file_path):

    ## import advertiser from google sheet
    SHEET_ID = 'SHEET_ID'
    SHEET_NAME = 'SHEET_NAME'
    url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
    adver = pd.read_csv(url)

    agencies = {
        'agency01': {
            'business_id': '',
            'agency_id': '',
            'agency': '',
        },
        'agency02': {
            'business_id': '',
            'agency_id': '',
            'agency': '',
        },
    }
  
    for agency_name, agency_data in agencies.items():
        if agency_name in file_path:
            business_id = agency_data['business_id']
            agency_id = agency_data['agency_id']
            agency = agency_data['agency']
        
        df = pd.read_excel(file_path)
        
        if 'agegender' in file_path:
            col_check = ['Account ID', 'Account name', 'Campaign ID', 'Campaign name', 'Month', 'Gender', 'Age', 'Ad set ID', 'Ad set name', 'Ad ID', 'Ad name', 'Impressions', 'Currency', 'Amount spent', 'Link clicks', 'Page likes', '3-second video plays', 'Landing page views', 'Objective', 'App Installs', 'Meta leads', 'Mobile app adds to cart', 'Website adds to cart', 'Mobile app content views', 'Website content views', 'Mobile app purchases', 'Website purchases', 'Mobile app adds to cart conversion value', 'Website adds to cart conversion value', 'Mobile app purchases conversion value', 'Website purchases conversion value', 'Leads', 'Clicks (all)', 'Post engagement', 'Reporting starts', 'Reporting ends']
        elif 'performance' in file_path:
            col_check = ['Account ID', 'Account name', 'Campaign ID', 'Campaign name', 'Month', 'Platform', 'Device platform', 'Placement', 'Ad set ID', 'Ad set name', 'Ad ID', 'Ad name', 'Impressions', 'Currency', 'Amount spent', 'Link clicks', 'Page likes', '3-second video plays', 'Landing page views', 'Objective', 'App Installs', 'Meta leads', 'Mobile app adds to cart', 'Website adds to cart', 'Mobile app content views', 'Website content views', 'Mobile app purchases', 'Website purchases', 'Mobile app adds to cart conversion value', 'Website adds to cart conversion value', 'Mobile app purchases conversion value', 'Website purchases conversion value', 'Leads', 'Clicks (all)', 'Post engagement', 'Reporting starts', 'Reporting ends']

        if 'Amount spent (THB)' in df.columns:
            df.rename(columns={'Amount spent (THB)': 'Amount spent'}, inplace=True)
        elif 'landing page view' in df.columns:
            df.rename(columns={'landing page view': 'Landing page views'}, inplace=True)

        # Check if file and variable have same columns
        if set(col_check).issubset(df.columns):
            extract_year = df['Month'].str.extract("([0-9]{4})")
            df = df[col_check]
            df.drop(df.columns[-2:], axis=1, inplace=True)
            df.insert(0, 'Business ID', business_id)
            df.insert(1, 'Media Platform', 'Facebook')
            df.insert(2, 'Agency ID', agency_id)
            df.insert(3, 'Agency', agency)
            df.insert(4, 'Year', extract_year)
            df['Month'] = df['Month'].str[5:7].astype(str).apply(lambda x: '0'+x if len(x) == 1 else x )

            ## Join data with google sheet
            df = pd.merge(df, adver, how='left', on='Account name')

            ## rename to lower and replace white spaces with underscore
            df.columns = map(str.lower, df.columns)
            df.columns = df.columns.str.replace(' ', '_')

            # 2 ifs are for changing data type
        if 'agegender' in file_path:
            col_str = ['business_id', 'media_platform', 'agency_id', 'agency', 'year', 'account_id', 'account_name', 'campaign_id', 'campaign_name', 'month', 'gender', 'age',
                        'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name', 'currency', 'objective']
            col_float = ['impressions', 'amount_spent', 'link_clicks', 'page_likes', '3-second_video_plays', 'landing_page_views', 'app_installs', 'meta_leads', 'mobile_app_adds_to_cart', 
                        'website_adds_to_cart', 'mobile_app_content_views', 'website_content_views', 'mobile_app_purchases', 'website_purchases', 'mobile_app_adds_to_cart_conversion_value', 
                        'website_adds_to_cart_conversion_value', 'mobile_app_purchases_conversion_value', 'website_purchases_conversion_value', 'leads', 'clicks_(all)', 'post_engagement']
            df[col_str] = df[col_str].astype(str).apply(lambda x: x.str.rstrip('.0'))
            df[col_float] = df[col_float].astype(float)

            df_ag = pd.concat([df_ag, df])

            ## file name
            blob_name = 'fb_agegender_final.parquet'

            ## export file to file path (/tmp/{file_name})
            df.to_parquet(file_path ,index=False)

            storage_client = storage.Client()
            bucket_name = 'bucket_name'
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            ## Upload to destination storage
            blob.upload_from_filename(file_path, content_type='application/octet-stream')

        elif 'performance' in file_path:
            col_str = ['business_id', 'media_platform', 'agency_id', 'agency', 'year', 'account_id', 'account_name', 'campaign_id', 'campaign_name', 'month',
                        'platform', 'device_platform', 'placement', 'ad_set_id', 'ad_set_name', 'ad_id', 'ad_name', 'currency', 'objective']
            col_float = ['impressions', 'amount_spent', 'link_clicks', 'page_likes', '3-second_video_plays', 'landing_page_views', 'app_installs', 'meta_leads', 'mobile_app_adds_to_cart', 
                        'website_adds_to_cart', 'mobile_app_content_views', 'website_content_views', 'mobile_app_purchases', 'website_purchases', 'mobile_app_adds_to_cart_conversion_value', 
                        'website_adds_to_cart_conversion_value', 'mobile_app_purchases_conversion_value', 'website_purchases_conversion_value', 'leads', 'clicks_(all)', 'post_engagement']
            df[col_str] = df[col_str].astype(str).apply(lambda x: x.str.rstrip('.0'))
            df[col_float] = df[col_float].astype(float)

            df_dpp = pd.concat([df_dpp, df])

            ## file name
            blob_name = 'fb_dpp_final.parquet'

            ## export file to file path (/tmp/{file_name})
            df.to_parquet(file_path ,index=False)

            storage_client = storage.Client()
            bucket_name = 'bucket_name'
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            ## Upload to destination storage
            blob.upload_from_filename(file_path, content_type='application/octet-stream')

In [ ]:
## dv360

import pandas as pd
from google.cloud import storage
from datetime import datetime, timedelta
import os

def prep_data_csv(event, context):
    bucket_name = ''
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    for blob in blobs:
        if blob.name.endswith('.csv'):
            file_name = os.path.basename(blob.name)
            modified_time = blob.updated.replace(tzinfo=None)

            ## Get the latest file only
            if modified_time >= datetime.now() - timedelta(hours=24):
                tmp_path = f"/tmp/{file_name}"
                download_blob(bucket_name, file_name, tmp_path)

def download_blob(bucket_name, blob_name, tmp_path,request=None):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(tmp_path)
    clean_upload_data(tmp_path)

def clean_upload_data(file_path):
    agencies = {
        'agency1': {
        'agency': '',
        'agency_id': '',
        },
        'agency2': {
        'agency': '',
        'agency_id': '',
        },
        'agency3': {
        'agency': '',
        'agency_id': '',
        }
    }

    ## Import advertiser data from google sheet
    SHEET_ID = ''
    SHEET_NAME = ''
    url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
    adver = pd.read_csv(url)

    replace_word = {'word1': 'w1', 'word2': 'w2', 'word3': 'w3', 'word4': 'w4', 'word5': 'w5'}
    col_check = ['Partner ID', 'Partner', 'Advertiser ID', 'Advertiser', 'Advertiser Currency', 'Campaign ID', 'Campaign', 'Insertion Order ID', 'Insertion Order', 'Line Item ID', 'Line Item', 
                'Year', 'Month', 'Bid Strategy Type', 'Creative Type', 'Device Type', 'Media Type', 'Impressions', 'Clicks', 'Media Cost (Advertiser Currency)', 'Total Media Cost (Advertiser Currency)', 
                'First-Quartile Views (Video)', 'Midpoint Views (Video)', 'Third-Quartile Views (Video)', 'Complete Views (Video)', 'Active View: Viewable Impressions']

    df = pd.read_csv(file_path)

    if len(df.columns.tolist()) == 28:
        if set(col_check).issubset(df.columns):

            df.drop(df.tail(25).index, inplace=True)
            df.insert(0, 'Media Platform', 'DV360')
            df.rename(columns={'Partner ID': 'Business ID', 'Partner': 'Business','Advertiser': 'Account name', 'Advertiser ID': 'Account ID'}, inplace=True)
            df['Agency'] = df['Account name'].str.extract(r"(?<=THB\)(word1|word2|word3|word4|word5)").replace(replace_word, regex=True)
            df['Agency ID'] = df['Agency'].str.lower().map({agency['agency'].lower(): agency['agency_id'] for agency in agencies.values()})
            df['Month'] = df['Month'].str.extract("/([0-9]{2})")

            ## join data
            df = pd.merge(df, adver, how= "left", left_on= "Account name", right_on= "Account name")

            df.columns = map(str.lower, df.columns)
            df.columns = df.columns.str.replace(' ', '_')

            ## change data type
            col_to_str = ['media_platform', 'business_id', 'business', 'agency_id', 'agency', 'advertiser', 'industry', 'account_id', 'account_name', 'advertiser_currency', 'campaign_id', 'campaign',
                        'insertion_order_id', 'insertion_order', 'line_item_id', 'line_item', 'app/url_id', 'app/url', 'year', 'month', 'bid_strategy_type', 'creative_type', 'device_type', 'media_type']
            col_to_float = ['impressions', 'clicks', 'media_cost_(advertiser_currency)', 'total_media_cost_(advertiser_currency)', 'first-quartile_views_(video)', 'midpoint_views_(video)', 'third-quartile_views_(video)',
                            'complete_views_(video)', 'active_view:_viewable_impressions']
            
            df[col_to_str] = df[col_to_str].astype(str)
            df[col_to_float] = df[col_to_float].astype(float)

            ## save to parquet file
            df.to_parquet(file_path, index= False)

            blob_name = os.path.basename(file_path).replace('.csv', '.parquet')
            storage_client = storage.Client()
            bucket_name = ''
            bucket = storage_client.bucket(bucket_name)
            blob = bucket.blob(blob_name)

            # Upload to destination storage
            blob.upload_from_filename(file_path, content_type='application/octet-stream')

        else:
            print(f"{file_path} check column name")
    else:
        print(f"{file_path} check amount of column")

In [ ]:
## TikTok

import pandas as pd
from google.cloud import storage
from datetime import datetime, timedelta
import os

def prep_data_csv(event, context):
    bucket_name = ''
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blobs = bucket.list_blobs()

    for blob in blobs:
        if blob.name.endswith('.csv'):
            file_name = os.path.basename(blob.name)
            modified_time = blob.updated.replace(tzinfo=None)

            ## Get the latest file only
            if modified_time >= datetime.now() - timedelta(hours=24):
                tmp_path = f"/tmp/{file_name}"
                download_blob(bucket_name, file_name, tmp_path)

def download_blob(bucket_name, blob_name, tmp_path, request=None):
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(tmp_path)
    clean_upload_data(tmp_path)

def clean_upload_data(file_path):
    agencies = [
        {'agency': '', 'agency_id': ''},
        {'agency': '', 'agency_id': ''},
        {'agency': '', 'agency_id': ''}
    ]

    month = {
    'January': '01',
    'February': '02',
    'March': '03',
    'April': '04',
    'May': '05',
    'June': '06',
    'July': '07',
    'August': '08',
    'September': '09',
    'October': '10',
    'November': '11',
    'December': '12',
    }

    sheet_id = ''
    sheet_name = ''
    url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df_adver = pd.read_csv(url)

    df = pd.read_csv(file_path)
    
    if 'agegender' in file_path:
        col_check = ['Year', 'Month', 'Advertiser ID', 'Advertiser name', 'Campaign ID', 'Campaign name', 'Objective', 'Adgroup ID', 'Adgroup name', 'Age', 'Gender', 'Bid Type', 'Placement Type', 'Currency',
                    'Cost', 'Impressions', 'Clicks', 'Conversions', '2s VDO', '6s VDO', 'Video Views', 'Video Views at 25%', 'Video Views at 50%', 'Video Views at 75%', 'Video Views at 100%', 'Complete Views']

        if set(col_check).issubset(df.columns) and len(df.columns.tolist()) == len(col_check):
            df.rename(columns={'Advertiser ID': 'Account ID', 'Advertiser name': 'Account name'}, inplace=True)
            df = df.merge(df_adver, on='Account name', how='left')
            df.insert(0, 'Media Platform', 'TikTok')
            df.insert(1, 'Agency ID', df['Agency'].map(dict((d['agency'], d['agency_id']) for d in agencies)))
            df.insert(2, 'Agency', df.pop('Agency'))
            df['Month'] = df['Month'].map(month)

            df.columns = map(str.lower, df.columns)
            df.columns = df.columns.str.replace(' ', '_')

            ## change data type
            col_to_str = ['year', 'month', 'advertiser_id', 'advertiser_name', 'campaign_id', 'campaign_name', 'objective', 'adgroup_id', 'adgroup_name', 'age', 'gender', 'bid_type', 'placement_type', 'currency']
            col_to_float = ['cost', 'impressions', 'clicks', 'conversions', '2s_vdo', '6s_vdo', 'video_views', 'video_views_at_25%', 'video_views_at_50%', 'video_views_at_75%', 'video_views_at_100%', 'complete_views']
      
            df[col_to_str] = df[col_to_str].astype(str)
            df[col_to_float] = df[col_to_float].astype(float)

            df.to_parquet(file_path, index=False)

    if 'device' in file_path:
        col_check = ['Year', 'Month', 'Advertiser ID', 'Advertiser name', 'Campaign ID', 'Campaign name', 'Objective', 'Adgroup ID', 'Adgroup name', 'Device Brand', 'Bid Type', 'Placement Type', 'Currency', 
                    'Cost', 'Impressions', 'Clicks', 'Conversions', '2s VDO', '6s VDO', 'Video Views', 'Video Views at 25%', 'Video Views at 50%', 'Video Views at 75%', 'Video Views at 100%', 'Complete Views']
    
        if set(col_check).issubset(df.columns) and len(df.columns.tolist()) == len(col_check):
            df.rename(columns={'Advertiser ID': 'Account ID', 'Advertiser name': 'Account name'}, inplace=True)
            df = df.merge(df_adver, on='Account name', how='left')
            df.insert(0, 'Media Platform', 'TikTok')
            df.insert(1, 'Agency ID', df['Agency'].map(dict((d['agency'], d['agency_id']) for d in agencies)))
            df.insert(2, 'Agency', df.pop('Agency'))
            df['Month'] = df['Month'].map(month)

            df.columns = map(str.lower, df.columns)
            df.columns = df.columns.str.replace(' ', '_')

            ## change data type
            col_to_str = ['year', 'month', 'advertiser_id', 'advertiser_name', 'campaign_id', 'campaign_name', 'objective', 'adgroup_id', 'adgroup_name', 'device_brand', 'bid_type', 'placement_type', 'currency']
            col_to_float = ['cost', 'impressions', 'clicks', 'conversions', '2s_vdo', '6s_vdo', 'video_views', 'video_views_at_25%', 'video_views_at_50%', 'video_views_at_75%', 'video_views_at_100%', 'complete_views']
            
            df[col_to_str] = df[col_to_str].astype(str)
            df[col_to_float] = df[col_to_float].astype(float)

            df.to_parquet(file_path, index=False)

    blob_name = os.path.basename(file_path).replace('.csv', '.parquet')
    storage_client = storage.Client()
    bucket_name = ''
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(blob_name)

    ## Upload to destination storage
    blob.upload_from_filename(file_path, content_type='application/octet-stream')